# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('combined_time.npz') 

# Access the concatenated features from the .npz file
X_time = data['X_time']

# Close the loaded file
data.close()

#print the merged array
X_time

array([[[ 4.13828693e-06,  3.41978478e-06,  3.05983596e-11, ...,
          3.05983596e-11,  5.69918532e-01,  1.43285998e+00],
        [-6.11778468e-07, -1.07570082e-06,  1.80793891e-11, ...,
          1.80793891e-11,  3.16028756e-01,  2.11285116e+00],
        [-1.40264838e-06, -1.29630615e-06,  2.03574174e-11, ...,
          2.03574174e-11,  3.68552716e-01,  1.69556851e+00],
        ...,
        [-5.92285647e-07, -5.51680372e-07,  2.44995562e-11, ...,
          2.44995562e-11,  3.34763239e-01,  2.11152035e+00],
        [-1.31523003e-06, -4.78786390e-07,  2.61209914e-11, ...,
          2.61209914e-11,  3.37369781e-01,  1.97140856e+00],
        [ 1.51803122e-06,  1.54910356e-06,  7.24925208e-11, ...,
          7.24925208e-11,  4.56258399e-01,  1.62313244e+00]],

       [[-6.12151008e-06, -9.22272966e-06,  1.79259348e-10, ...,
          1.79259348e-10,  2.21715280e-01,  3.09942440e+00],
        [-5.45245164e-07, -8.51270215e-07,  3.87428576e-11, ...,
          3.87428576e-11,  2.51679355e

In [3]:
data = np.load('combined_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_time.shape, Y.shape, groups_array.shape)

(73519, 20, 13) (73519,) (73519,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention, Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=512, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Bidirectional(LSTM(256)))

    model.add(Flatten())

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam
import os
import csv

# Define lists to store evaluation metrics for each fold
metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_time, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_time[train_index], X_time[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

     # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=100, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks = [tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/100
259/259 [==============================] - 9s 21ms/step - loss: 0.2358 - accuracy: 0.8969 - val_loss: 0.1523 - val_accuracy: 0.9411
Epoch 2/100
259/259 [==============================] - 5s 18ms/step - loss: 0.1408 - accuracy: 0.9448 - val_loss: 0.0928 - val_accuracy: 0.9671
Epoch 3/100
259/259 [==============================] - 5s 18ms/step - loss: 0.1079 - accuracy: 0.9588 - val_loss: 0.0766 - val_accuracy: 0.9714
Epoch 4/100
259/259 [==============================] - 4s 17ms/step - loss: 0.0887 - accuracy: 0.9668 - val_loss: 0.0736 - val_accuracy: 0.9733
Epoch 5/100
259/259 [==============================] - 4s 17ms/step - loss: 0.0781 - accuracy: 0.9713 - val_loss: 0.0552 - val_accuracy: 0.9810
Epoch 6/100
259/259 [==============================] - 4s 17ms/step - loss: 0.0680 - accuracy: 0.9752 - val_loss: 0.0558 - val_accuracy: 0.9804
Epoch 7/100
259/259 [==============================] - 5s 18ms/step - loss: 0.0614 - accuracy: 0.9779 - val_loss: 0.0438 - val_ac

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9950,0.9949,0.9957,0.9953,0.9949,0.9942,0.9957,0.9899,0.9899
1,0.9944,0.9946,0.9949,0.9948,0.9944,0.9939,0.9949,0.9888,0.9888
2,0.9951,0.9949,0.9959,0.9954,0.9950,0.9942,0.9959,0.9902,0.9902
3,0.9947,0.9959,0.9941,0.9950,0.9947,0.9953,0.9941,0.9893,0.9893
4,0.9956,0.9954,0.9964,0.9959,0.9956,0.9948,0.9964,0.9913,0.9913
5,0.9942,0.9916,0.9974,0.9945,0.9939,0.9904,0.9974,0.9882,0.9883
6,0.9937,0.9949,0.9934,0.9941,0.9938,0.9942,0.9934,0.9874,0.9874
7,0.9942,0.9946,0.9944,0.9945,0.9941,0.9939,0.9944,0.9883,0.9883
8,0.9959,0.9974,0.9949,0.9962,0.9960,0.9971,0.9949,0.9918,0.9918
9,0.9943,0.9959,0.9934,0.9946,0.9944,0.9953,0.9934,0.9885,0.9885


In [13]:
metrics_df.round(4).to_csv('Four+LSTM_TIME_COMBINED_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("Four+LSTM_TIME_COMBINED_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9950,0.9949,0.9957,0.9953,0.9949,0.9942,0.9957,0.9899,0.9899
1,0.9944,0.9946,0.9949,0.9948,0.9944,0.9939,0.9949,0.9888,0.9888
2,0.9951,0.9949,0.9959,0.9954,0.9950,0.9942,0.9959,0.9902,0.9902
3,0.9947,0.9959,0.9941,0.9950,0.9947,0.9953,0.9941,0.9893,0.9893
4,0.9956,0.9954,0.9964,0.9959,0.9956,0.9948,0.9964,0.9913,0.9913
5,0.9942,0.9916,0.9974,0.9945,0.9939,0.9904,0.9974,0.9882,0.9883
6,0.9937,0.9949,0.9934,0.9941,0.9938,0.9942,0.9934,0.9874,0.9874
7,0.9942,0.9946,0.9944,0.9945,0.9941,0.9939,0.9944,0.9883,0.9883
8,0.9959,0.9974,0.9949,0.9962,0.9960,0.9971,0.9949,0.9918,0.9918
9,0.9943,0.9959,0.9934,0.9946,0.9944,0.9953,0.9934,0.9885,0.9885


In [16]:
(result.mean()*100).round(2)

Accuracy       99.47
Precision      99.50
Recall         99.50
F1             99.50
Sensitivity    99.47
Specificity    99.43
ROC_AUC        99.50
Kappa          98.94
MCC            98.94
dtype: float64

In [19]:
import csv
for i in range (1, 11):
    df = pd.read_csv(f'epoch_{i}.csv')
    [col] = df.columns
    split_data = df[col].str.split(';')
    csv_file = (f'epoch_{i}.csv')
    # Prepare CSV file and write headers
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
    for j in range (0, 100):
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])